In [2]:
# Import dependencies
import requests
import json
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
import time

# Load API Key
load_dotenv(find_dotenv('api.env')) # api.env not included in github. Please make separate env file.
api_key = os.getenv("POLYGON_KEY")

df = pd.DataFrame() # Create Empty Dataframe

base_url = "https://api.polygon.io"
stocksTicker = f"SPY" # SPY = S&P 500 <-- Change this for different stock

for od in range(240318, 240323): # Dates range, Change as necessary
    for strk in range (480, 521): # Strike range, Change as necessary
        odate = str(od)
        strike = str(strk)
        optionsTicker = f"O:{stocksTicker}{odate}C00{strike}000" # This assumes strike is 3-digit. remove a 0 in front if it's 4-digit
        multiplier = 5
        timespan = 'minute' # multiplier + timespan = 5 minutes
        test = f"20{odate[:2]}-{odate[2:4]}-{odate[4:]}"

        query_url = f"{base_url}/v2/aggs/ticker/{optionsTicker}/range/{multiplier}/{timespan}/{test}/{test}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"
        # Get the response
        response = requests.get(query_url)

        # Retrieve reviews
        reviews = response.json()

        if reviews['resultsCount'] > 0:
            # Convert JSON to Pandas Dataframe
            reviews_df = pd.json_normalize(reviews["results"])
            reviews_df = reviews_df.rename(columns={'v':'Volume', 'vw':'Volume Weighted', 'o':'Open Price', 'c':'Close Price', 'h':'Highest Price', 'l':'Lowest Price', 't':'Timestamp', 'n':'Number of Transaction'})
            reviews_df['Timestamp'] = pd.to_datetime(reviews_df['Timestamp'], unit='ms')
            reviews_df['Stock Name'] = stocksTicker
            reviews_df['Strike'] = strike

            df = pd.concat([df, reviews_df], ignore_index= True, axis = 0)

In [3]:
df

,Volume,Volume Weighted,Open Price,Close Price,Highest Price,Lowest Price,Timestamp,Number of Transaction,Stock Name,Strike
0,1,32.5100,32.51,32.51,32.51,32.51,2024-03-18 17:55:00,1,SPY,481
1,9,29.8600,29.86,29.86,29.86,29.86,2024-03-18 19:55:00,2,SPY,483
2,2,27.7500,27.75,27.75,27.75,27.75,2024-03-18 17:55:00,1,SPY,486
3,8,26.7500,26.75,26.75,26.75,26.75,2024-03-18 17:55:00,1,SPY,487
4,34,24.9100,24.91,24.91,24.91,24.91,2024-03-18 19:55:00,2,SPY,488
...,...,...,...,...,...,...,...,...,...,...
4869,339,1.7970,1.60,1.81,1.89,1.57,2024-03-22 19:35:00,29,SPY,520
4870,177,1.8690,1.84,1.97,2.02,1.75,2024-03-22 19:40:00,37,SPY,520
4871,217,1.9543,1.95,2.02,2.06,1.84,2024-03-22 19:45:00,42,SPY,520
4872,424,2.2132,2.05,2.21,2.34,1.97,2024-03-22 19:50:00,55,SPY,520


In [5]:
#Save data to CSV
df.to_csv('../Data/ospy-agg-03-22-03-22.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)
# -------------------------^^^ Change this to stock you chose

In [10]:
# Test CSV Import

df = pd.read_csv('../Data/ospy-agg-03-22-03-22.csv', parse_dates=['Timestamp'])
# ------------------------^^^ Change this to stock you chose
df

,Volume,Volume Weighted,Open Price,Close Price,Highest Price,Lowest Price,Timestamp,Number of Transaction,Stock Name,Strike
4814,3274,1.7202,1.46,1.73,1.83,1.46,2024-03-22 15:00:00,181,SPY,520
4815,936,1.7077,1.73,1.65,1.82,1.59,2024-03-22 15:05:00,93,SPY,520
4816,947,1.7835,1.69,1.75,1.86,1.69,2024-03-22 15:10:00,50,SPY,520
4817,240,1.6736,1.75,1.70,1.76,1.61,2024-03-22 15:15:00,40,SPY,520
4818,1010,1.8339,1.71,1.85,1.88,1.71,2024-03-22 15:20:00,55,SPY,520
4819,389,1.8712,1.87,1.89,1.95,1.77,2024-03-22 15:25:00,69,SPY,520
4820,743,1.9529,1.93,1.99,2.02,1.91,2024-03-22 15:30:00,60,SPY,520
4821,1185,1.9289,1.97,1.70,2.02,1.65,2024-03-22 15:35:00,84,SPY,520
4822,865,1.7765,1.76,1.70,1.85,1.70,2024-03-22 15:40:00,21,SPY,520
4823,158,1.7334,1.65,1.69,1.81,1.65,2024-03-22 15:45:00,34,SPY,520


In [7]:
# Analysis - Checking Max values
print(f"Max values for Volume: {df['Volume'].max()}")
print(f"Max values for Volume Weighted: {df['Volume Weighted'].max()}")
print(f"Max values for Open Price: {df['Open Price'].max()}")
print(f"Max values for Close Price: {df['Close Price'].max()}")
print(f"Max values for Highest Price: {df['Highest Price'].max()}")
print(f"Max values for Lowest Price: {df['Lowest Price'].max()}")
print(f"Max values for Number of Transaction: {df['Number of Transaction'].max()}")

# Looking at the result, standardizing of data will be needed!

Max values for Volume: 17778
Max values for Volume Weighted: 43.92
Max values for Open Price: 43.92
Max values for Close Price: 43.92
Max values for Highest Price: 43.92
Max values for Lowest Price: 43.92
Max values for Number of Transaction: 2966
